# BME 590 Final Project - if 你想加个 title (30-Day Readmission)
### Name: Cindy Wang, Siyan He
### NetID: rw193, sh431


第一列 病人id 第二列 intercept 全1 然后每种疾病类别 1/0 最后一列 是否30天内readmitted

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import seaborn as sns
import textwrap
import math

## Background

## Data

In [2]:
admission_df = pd.read_csv("./data/ADMISSIONS.csv")
admission_df

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,58594,98800,191113,2131-03-30 21:13:00,2131-04-02 15:02:00,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,2131-03-30 19:44:00,2131-03-30 22:41:00,TRAUMA,0,1
58972,58595,98802,101071,2151-03-05 20:00:00,2151-03-06 09:10:00,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2151-03-05 17:23:00,2151-03-05 21:06:00,SAH,1,1
58973,58596,98805,122631,2200-09-12 07:15:00,2200-09-20 12:08:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,NOT SPECIFIED,MARRIED,WHITE,NaN,NaN,RENAL CANCER/SDA,0,1
58974,58597,98813,170407,2128-11-11 02:29:00,2128-12-22 13:11:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Private,ENGL,CATHOLIC,MARRIED,WHITE,2128-11-10 23:48:00,2128-11-11 03:16:00,S/P FALL,0,0


In [3]:
diagnoses_df = pd.read_csv("./data/DIAGNOSES_ICD.csv")
diagnoses_df

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [4]:
#length between admissions
data_df = admission_df.loc[:,['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DIAGNOSIS']].sort_values('SUBJECT_ID', ascending=True)
data_df[['ADMITTIME','DISCHTIME']] = data_df[['ADMITTIME','DISCHTIME']].apply(lambda x: pd.to_datetime(x))
readmt_df = data_df.sort_values(['SUBJECT_ID','ADMITTIME'])

readmt_df['READMISSION_30']=readmt_df.groupby('SUBJECT_ID').apply(lambda x : (x['ADMITTIME'].shift(-1)-x['DISCHTIME']).dt.days.le(30).astype(int)).reset_index(0).drop('SUBJECT_ID',1)
readmt_df

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DIAGNOSIS,READMISSION_30
211,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NEWBORN,0
212,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,HYPOTENSION,0
213,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",0
214,5,178980,2103-02-02 04:31:00,2103-02-04 12:15:00,NEWBORN,0
215,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,CHRONIC RENAL FAILURE/SDA,0
...,...,...,...,...,...,...
56435,99985,176670,2181-01-27 02:47:00,2181-02-12 17:05:00,FEVER,0
56436,99991,151118,2184-12-24 08:30:00,2185-01-05 12:15:00,DIVERTICULITIS/SDA,0
56437,99992,197084,2144-07-25 18:03:00,2144-07-28 17:56:00,RETROPERITONEAL HEMORRHAGE,0
56565,99995,137810,2147-02-08 08:00:00,2147-02-11 13:15:00,ABDOMINAL AORTIC ANEURYSM/SDA,0


In [5]:
#merge code
digns_df = pd.merge(data_df, diagnoses_df, how='inner', on=['SUBJECT_ID', 'HADM_ID']).loc[:,['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DIAGNOSIS', 'ICD9_CODE']]
digns_df


,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DIAGNOSIS,ICD9_CODE
0,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NEWBORN,V3001
1,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NEWBORN,V053
2,2,163353,2138-07-17 19:04:00,2138-07-21 15:48:00,NEWBORN,V290
3,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,HYPOTENSION,0389
4,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,HYPOTENSION,78559
...,...,...,...,...,...,...
651042,99999,113369,2117-12-30 07:15:00,2118-01-04 16:30:00,SPONDYLOLISTHESIS/SDA,75612
651043,99999,113369,2117-12-30 07:15:00,2118-01-04 16:30:00,SPONDYLOLISTHESIS/SDA,7861
651044,99999,113369,2117-12-30 07:15:00,2118-01-04 16:30:00,SPONDYLOLISTHESIS/SDA,4019
651045,99999,113369,2117-12-30 07:15:00,2118-01-04 16:30:00,SPONDYLOLISTHESIS/SDA,25000


In [6]:
#unstack cat.
digns_df_wide = digns_df.groupby(['HADM_ID', 'DIAGNOSIS']).size().unstack()
digns_df_wide

DIAGNOSIS,MITRAL STENOSIS\MITRAL VALVE REPLACEMENT /SDA,DUODENAL MASS/SDA,? SEROTONIN SYNDROME,ABDOMINAL AORTIC ANEURYSM/SDA,ABSENCE/SDA,ADENOID CYSTIC CARCINOMA/SDA,ANALOMOUS PULMANARY VEIN\ REPAIR ANALOMOUS ; PULMONARY VEIN/SDA,AORTIC ABDOMINAL ANEURYSM/SDA,AORTIC STENOSIS\AORTIC VALVE REPLACEMENT/SDA,CAD\CORONARY ARTERY BYPASS GRAFT/SDA,...,WOUND INFECTION R/O SEPSIS,WOUND INFECTION'OSTEOMYELITIS,WOUND INFECTION;HYPOTENSION,WOUND INFECTION;TELEMETRY,WOUND TO RLE,WOUND ULCER,ZENKER'S DIVERTICULUM/SDA,ZONE 2 NECK INJURY,ZOSTER VS. MENINGITIS,ZYGOMATIC FRACTURE LEFT
HADM_ID,,,,,,,,,,,,,,,,,,,,,
100001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#binary
digns_df_wide.fillna(0, inplace=True)
digns_df_wide[digns_df_wide > 0] = 1
digns_df_wide


DIAGNOSIS,MITRAL STENOSIS\MITRAL VALVE REPLACEMENT /SDA,DUODENAL MASS/SDA,? SEROTONIN SYNDROME,ABDOMINAL AORTIC ANEURYSM/SDA,ABSENCE/SDA,ADENOID CYSTIC CARCINOMA/SDA,ANALOMOUS PULMANARY VEIN\ REPAIR ANALOMOUS ; PULMONARY VEIN/SDA,AORTIC ABDOMINAL ANEURYSM/SDA,AORTIC STENOSIS\AORTIC VALVE REPLACEMENT/SDA,CAD\CORONARY ARTERY BYPASS GRAFT/SDA,...,WOUND INFECTION R/O SEPSIS,WOUND INFECTION'OSTEOMYELITIS,WOUND INFECTION;HYPOTENSION,WOUND INFECTION;TELEMETRY,WOUND TO RLE,WOUND ULCER,ZENKER'S DIVERTICULUM/SDA,ZONE 2 NECK INJURY,ZOSTER VS. MENINGITIS,ZYGOMATIC FRACTURE LEFT
HADM_ID,,,,,,,,,,,,,,,,,,,,,
100001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#merge diagnoses and 30-readmit
merge_df = digns_df_wide.merge(readmt_df[['HADM_ID','READMISSION_30']], left_on = digns_df_wide.index, right_on='HADM_ID')

HADM_ID = merge_df['HADM_ID']
merge_df.drop(labels = ['HADM_ID'], axis = 1, inplace = True)
merge_df.insert(0, 'Intercept', HADM_ID)
merge_df['Intercept'] = 1.0

merge_df

## Exploratory Data Analysis

## Modeling

## Conclusion 